In [1]:
# 1. 랭스미스 초기화

import os
from dotenv import load_dotenv
from langchain.callbacks.manager import tracing_v2_enabled

load_dotenv()

# 트레이싱 상태 확인
with tracing_v2_enabled() as tracing:
    print(f"Tracing enabled: {tracing}")


Tracing enabled: <langchain_core.tracers.langchain.LangChainTracer object at 0x1102a0cd0>


/Users/joajoa/Desktop/Naver/langchain_demo_코드정리/new_joavenv/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
# 오리지날 실행 
# 체인 초기화 및 실행
from llm_compiler import initialize_chain
from langchain_core.messages import HumanMessage
from pprint import pprint

chain = initialize_chain()

# 3. 스트리밍 실행
for step in chain.stream(
    {"messages": [HumanMessage(content=" 매출 알려줘")]}
):
    print(step)
    print("---")

print("---------------------최종 답변 --------------------------")
print("------------------------------------------------------")
pprint(step["join"]["messages"][-1].content)
print("------------------------------------------------------")
print("------------------------------------------------------")

In [3]:
## Joiner = HyperClova 사용 결과 
#### 하이퍼 클로바 JOIN 사용 버전 실행 
from llm_compiler_hyper_join import initialize_chain
from langchain_core.messages import HumanMessage
from pprint import pprint

chain = initialize_chain()

# 3. 스트리밍 실행
for step in chain.stream(
    {"messages": [HumanMessage(content="현대차 영업이익률 알려줘")]}
):
    print(step)
    print("---")

print("---------------------최종 답변 --------------------------")
print("------------------------------------------------------")
pprint(step["join"]["messages"][-1].content)
print("------------------------------------------------------")
print("------------------------------------------------------")

기존 벡터스토어를 로드합니다...


/Users/joajoa/Desktop/Naver/langchain_demo_코드정리/new_joavenv/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(



=== 기업명 추출 시작 ===
입력 쿼리: 현대차 영업이익률((영업이익 / 매출액) × 100) 알려줘

[Step 1: LLM으로 회사명 텍스트 추출 중...]
LLM 응답: content='["현대차"]' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 478, 'total_tokens': 486, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0125-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-d7477126-5f3b-4b31-822e-bf23c9ecb654-0' usage_metadata={'input_tokens': 478, 'output_tokens': 8, 'total_tokens': 486, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
- LLM이 예상한 회사명 목록: ['현대차']

[Step 2: 현대차에 대한 유사 기업명 후보 추출 중...]
{'현대차증권': 0.7, '현대텍': 0.6666666666666667, '현대인': 0.6666666666666667, '대찬': 0.6333333333333333, '대창': 0.6333333333333333, '현대자동차': 0.

In [1]:
import asyncio
from langchain_core.messages import HumanMessage, FunctionMessage
from plan.join_test import structured_llm_call  # 여러분의 join 관련 함수들이 있는 모듈


In [2]:

async def test_join():
    messages = [
        HumanMessage(content='<companyName>현대자동차</companyName> 매출 알려줘'),
        FunctionMessage(
            content=str({
                'output': '현대자동차의 매출액은 다음과 같습니다:\n\n- **2023년 (제 56 기)**: 162,663,579,000,000 KRW\n- **2022년 (제 55 기)**: 142,527,535,000,000 KRW\n- **2021년 (제 54 기)**: 116,448,159,000,000 KRW\n\n매출액은 2021년부터 2023년까지 꾸준히 증가하고 있는 추세입니다.',
                'key_information': [{
                    'tool': '재무제표 도구',
                    'referenced_content': '현재 데이터에 따르면, 매출액은 다음과 같습니다:\n\n- **2023년 (제 56 기)**: 162,663,579,000,000 KRW\n- **2022년 (제 55 기)**: 142,527,535,000,000 KRW\n- **2021년 (제 54 기)**: 116,448,159,000,000 KRW',
                    'page_number': '연결 재무제표',
                    'filename': '현대자동차_2024_financial_statement'
                }]
            }),
            name='combined_financial_report_search',
            additional_kwargs={'idx': 1, 'args': {'query': '현대자동차 매출', 'company': '현대자동차', 'year': 2024}}
        ),
        FunctionMessage(
            content='join',
            name='join',
            additional_kwargs={'idx': 2, 'args': ()}
        )
    ]

    print("\n=== Join 테스트 시작 ===")
    result = await structured_llm_call(messages)
    print("\n=== 테스트 결과 ===")
    print(f"Thought: {result.thought}")
    print(f"Action Type: {result.action.type}")
    if hasattr(result.action, 'response'):
        print(f"Response: {result.action.response}")
    elif hasattr(result.action, 'feedback'):
        print(f"Feedback: {result.action.feedback}")

In [3]:
await test_join()


=== Join 테스트 시작 ===

=== structured_llm_call 시작 ===
입력 메시지: [HumanMessage(content='<companyName>현대자동차</companyName> 매출 알려줘', additional_kwargs={}, response_metadata={}), FunctionMessage(content="{'output': '현대자동차의 매출액은 다음과 같습니다:\\n\\n- **2023년 (제 56 기)**: 162,663,579,000,000 KRW\\n- **2022년 (제 55 기)**: 142,527,535,000,000 KRW\\n- **2021년 (제 54 기)**: 116,448,159,000,000 KRW\\n\\n매출액은 2021년부터 2023년까지 꾸준히 증가하고 있는 추세입니다.', 'key_information': [{'tool': '재무제표 도구', 'referenced_content': '현재 데이터에 따르면, 매출액은 다음과 같습니다:\\n\\n- **2023년 (제 56 기)**: 162,663,579,000,000 KRW\\n- **2022년 (제 55 기)**: 142,527,535,000,000 KRW\\n- **2021년 (제 54 기)**: 116,448,159,000,000 KRW', 'page_number': '연결 재무제표', 'filename': '현대자동차_2024_financial_statement'}]}", additional_kwargs={'idx': 1, 'args': {'query': '현대자동차 매출', 'company': '현대자동차', 'year': 2024}}, response_metadata={}, name='combined_financial_report_search'), FunctionMessage(content='join', additional_kwargs={'idx': 2, 'args': ()}, response_metadata={}, name=